In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('energydata_complete.csv')

In [3]:
df.head()

date  Appliances  lights     T1       RH_1    T2       RH_2  \
0  2016-01-11 17:00:00          60      30  19.89  47.596667  19.2  44.790000   
1  2016-01-11 17:10:00          60      30  19.89  46.693333  19.2  44.722500   
2  2016-01-11 17:20:00          50      30  19.89  46.300000  19.2  44.626667   
3  2016-01-11 17:30:00          50      40  19.89  46.066667  19.2  44.590000   
4  2016-01-11 17:40:00          60      40  19.89  46.333333  19.2  44.530000   

      T3       RH_3         T4  ...         T9   RH_9     T_out  Press_mm_hg  \
0  19.79  44.730000  19.000000  ...  17.033333  45.53  6.600000        733.5   
1  19.79  44.790000  19.000000  ...  17.066667  45.56  6.483333        733.6   
2  19.79  44.933333  18.926667  ...  17.000000  45.50  6.366667        733.7   
3  19.79  45.000000  18.890000  ...  17.000000  45.40  6.250000        733.8   
4  19.79  45.000000  18.890000  ...  17.000000  45.40  6.133333        733.9   

   RH_out  Windspeed  Visibility  Tdewpoint        rv1        rv2  
0    92.0   7.000000   63.000000        5.3  13.275433  13.275433  
1    92.0   6.666667   59.166667        5.2  18.606195  18.606195  
2    92.0   6.333333   55.333333        5.1  28.642668  28.642668  
3    92.0   6.000000   51.500000        5.0  45.410389  45.410389  
4    92.0   5.666667   47.666667        4.9  10.084097  10.084097  

[5 rows x 29 columns]

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19735 entries, 0 to 19734
Data columns (total 29 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         19735 non-null  object 
 1   Appliances   19735 non-null  int64  
 2   lights       19735 non-null  int64  
 3   T1           19735 non-null  float64
 4   RH_1         19735 non-null  float64
 5   T2           19735 non-null  float64
 6   RH_2         19735 non-null  float64
 7   T3           19735 non-null  float64
 8   RH_3         19735 non-null  float64
 9   T4           19735 non-null  float64
 10  RH_4         19735 non-null  float64
 11  T5           19735 non-null  float64
 12  RH_5         19735 non-null  float64
 13  T6           19735 non-null  float64
 14  RH_6         19735 non-null  float64
 15  T7           19735 non-null  float64
 16  RH_7         19735 non-null  float64
 17  T8           19735 non-null  float64
 18  RH_8         19735 non-null  float64
 19  T9  

In [5]:
df.isnull().sum()

date           0
Appliances     0
lights         0
T1             0
RH_1           0
T2             0
RH_2           0
T3             0
RH_3           0
T4             0
RH_4           0
T5             0
RH_5           0
T6             0
RH_6           0
T7             0
RH_7           0
T8             0
RH_8           0
T9             0
RH_9           0
T_out          0
Press_mm_hg    0
RH_out         0
Windspeed      0
Visibility     0
Tdewpoint      0
rv1            0
rv2            0
dtype: int64

In [6]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split


In [7]:
x = df[["T2"]]
y = df[["T6"]]

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [9]:
lm = LinearRegression()

In [10]:
lm.fit(x_train, y_train)

LinearRegression()

In [11]:
predictions = lm.predict(x_test)

In [12]:
from sklearn.metrics import r2_score

In [13]:
r_squared =  r2_score(y_test, predictions)

In [14]:
r_squared

0.6427789637197382

In [15]:
df.drop(['lights'], axis=1,inplace=True)
df.drop(['date'], axis=1, inplace=True)

In [16]:
df.shape

(19735, 27)

In [17]:
from sklearn.preprocessing import MinMaxScaler

In [18]:
scaler = MinMaxScaler()

In [19]:
norm_ft = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

In [20]:
ft = norm_ft.drop(['Appliances'], axis=1)
target_variable = norm_ft['Appliances']

In [21]:
x_train, x_test, y_train, y_test = train_test_split(ft, target_variable, test_size = 0.3, random_state = 42)

In [22]:
lr = LinearRegression()

In [23]:
lr.fit(x_train, y_train)

LinearRegression()

In [24]:
prediction = lr.predict(x_test)

In [25]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
m_a_e = mean_absolute_error(y_test, prediction)

In [26]:
round(m_a_e, 2)

0.05

In [27]:
r_s = np.sum(np.square(y_test - prediction))

In [28]:
round(r_s, 2)

45.35

In [29]:
r_m_s_e = np.sqrt(mean_squared_error(y_test, prediction))

In [30]:
round(r_m_s_e, 3)

0.088

In [31]:
coef_of_determination = r2_score(y_test, prediction)
round(coef_of_determination, 2)

0.15

In [32]:
def get_weights_df(model, feat, col_name):
  weights = pd.Series(model.coef_, feat.columns).sort_values()
  weights_df = pd.DataFrame(weights).reset_index()
  weights_df.columns = ['Features', col_name]
  weights_df[col_name].round(3)
  return weights_df

In [33]:
model_weight = get_weights_df(lr, x_train, 'Model_Weights')

In [34]:
model_weight

Features  Model_Weights
0          RH_2      -0.456698
1         T_out      -0.321860
2            T2      -0.236178
3            T9      -0.189941
4          RH_8      -0.157595
5        RH_out      -0.077671
6          RH_7      -0.044614
7          RH_9      -0.039800
8            T5      -0.015657
9            T1      -0.003281
10          rv2       0.000770
11          rv1       0.000770
12  Press_mm_hg       0.006839
13           T7       0.010319
14   Visibility       0.012307
15         RH_5       0.016006
16         RH_4       0.026386
17           T4       0.028981
18    Windspeed       0.029183
19         RH_6       0.038049
20         RH_3       0.096048
21           T8       0.101995
22    Tdewpoint       0.117758
23           T6       0.236425
24           T3       0.290627
25         RH_1       0.553547

In [35]:
ridge = Ridge(alpha=0.4)
ridge.fit(x_train, y_train)

Ridge(alpha=0.4)

In [36]:
lasso = Lasso(alpha=0.001)
lasso.fit(x_train, y_train)
pred = lasso.predict(x_test)

In [37]:
lasso_weight = get_weights_df(lasso, x_train, 'lasso_weight')

In [38]:
(lasso_weight['lasso_weight'] == 0).value_counts()

True     22
False     4
Name: lasso_weight, dtype: int64

In [39]:
r_m_s_e = np.sqrt(mean_squared_error(y_test, pred))

In [40]:
round(r_m_s_e, 3)

0.094